In [1]:
spark

In [2]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
import graphviz 
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pyspark.pandas as ps
import pyspark

In [3]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled",True)

In [4]:
ps.set_option("compute.default_index_type", "distributed")

In [5]:
df =  pd.read_csv("./Judgement_Feature1.csv")
df.head(10)

,judge_court,judge_year,judge_month,judge_index,judge_title,F_01,F_02,F_03,F_04,F_05,...,crimePredCategory,judge_crime,judge_crimeClassify,crimeCategory,Match,judge_result,judge_resultInt,judge_class,judge_class2,judge_class3
0,臺灣彰化地方法院,111,1,4,過失傷害,1,1,0,0,0,...,0,汽車駕駛人無駕駛執照駕車過失傷害罪,過失傷害,1,0,拘役參拾伍日,35,2,1,1
1,臺灣彰化地方法院,111,1,6,傷害致重傷害,0,2,0,0,0,...,4,傷害人致重傷罪,傷害致重傷,4,1,有期徒刑壹年捌月,605,4,1,3
2,臺灣彰化地方法院,111,1,18,重傷害,0,0,0,0,0,...,4,傷害罪,傷害,3,0,有期徒刑陸月,180,3,1,2
3,臺灣彰化地方法院,111,1,19,傷害,0,0,0,0,0,...,0,無罪,無罪,0,1,零日,0,0,0,0
4,臺灣彰化地方法院,111,1,27,傷害,0,0,0,0,0,...,0,無罪,無罪,0,1,零日,0,0,0,0
5,臺灣彰化地方法院,111,1,34,過失傷害,0,2,0,0,0,...,0,過失傷害罪,過失傷害,1,0,拘役拾伍日,15,1,1,1
6,臺灣彰化地方法院,111,1,36,過失傷害,0,2,0,0,0,...,9,過失傷害罪,過失傷害,1,0,有期徒刑參月,90,3,1,2
7,臺灣彰化地方法院,111,1,37,過失傷害,0,0,0,0,0,...,2,過失傷害罪,過失傷害,1,0,拘役伍拾日,50,2,1,1
8,臺灣彰化地方法院,111,1,39,過失傷害,0,0,0,0,0,...,2,過失傷害罪,過失傷害,1,0,有期徒刑參月,90,3,1,2
9,臺灣彰化地方法院,111,1,40,過失傷害,0,0,0,0,0,...,2,過失傷害罪,過失傷害,1,0,拘役伍拾捌日,58,2,1,1


In [6]:
y1 = df.loc[:,"judge_class"]    
y1_0 = df.where(y1==0)
y1_0 = y1_0.dropna()
y1_1 = df.where(y1==1)
y1_1 = y1_1.dropna()
y1_2 = df.where(y1==2)
y1_2 = y1_2.dropna()
y1_3 = df.where(y1==3)
y1_3 = y1_3.dropna()
y1_4 = df.where(y1==4)
y1_4 = y1_4.dropna()
y1_o = df.where(y1>4)
y1_o = y1_o.dropna()
y1_o

,judge_court,judge_year,judge_month,judge_index,judge_title,F_01,F_02,F_03,F_04,F_05,...,crimePredCategory,judge_crime,judge_crimeClassify,crimeCategory,Match,judge_result,judge_resultInt,judge_class,judge_class2,judge_class3
141,臺灣彰化地方法院,111.0,5.0,110.0,傷害致死,0.0,2.0,0.0,0.0,0.0,...,10.0,傷害致人於死罪,傷害致死,5.0,0.0,有期徒刑參年陸月,1275.0,5.0,1.0,4.0
469,臺灣彰化地方法院,110.0,9.0,26.0,傷害致死,0.0,2.0,0.0,0.0,0.0,...,10.0,傷害致人於死罪,傷害致死,5.0,0.0,有期徒刑伍年,1825.0,5.0,1.0,4.0
1205,臺灣彰化地方法院,107.0,9.0,47.0,傷害致重傷害等,1.0,0.0,0.0,0.0,0.0,...,4.0,傷害致人重傷罪,傷害致重傷,4.0,1.0,有期徒刑陸年,2190.0,6.0,1.0,5.0
1250,臺灣彰化地方法院,107.0,12.0,28.0,傷害致死,0.0,0.0,0.0,0.0,0.0,...,10.0,傷害致人於死罪,傷害致死,5.0,0.0,有期徒刑柒年貳月,2615.0,6.0,1.0,5.0
1389,臺灣彰化地方法院,106.0,9.0,17.0,重傷害等,0.0,2.0,1.0,0.0,0.0,...,6.0,重傷罪,重傷害,6.0,1.0,有期徒刑伍年貳月,1885.0,6.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64138,臺灣新北地方法院,103.0,3.0,150.0,重傷害等,1.0,0.0,0.0,0.0,0.0,...,10.0,傷害致人重傷罪,傷害致重傷,4.0,0.0,有期徒刑參年捌月,1335.0,5.0,1.0,4.0
64255,臺灣新北地方法院,103.0,5.0,187.0,家暴重傷害等,1.0,0.0,0.0,0.0,0.0,...,4.0,而致重傷,過失傷害致重傷,2.0,0.0,有期徒刑陸年捌月,2430.0,6.0,1.0,5.0
64655,臺灣新北地方法院,102.0,1.0,128.0,重傷害,0.0,0.0,0.0,1.0,0.0,...,4.0,傷害致人重傷罪,傷害致重傷,4.0,1.0,有期徒刑肆年,1460.0,5.0,1.0,4.0
64855,臺灣新北地方法院,102.0,5.0,198.0,重傷害等,1.0,0.0,1.0,0.0,0.0,...,4.0,而致重傷,過失傷害致重傷,2.0,0.0,有期徒刑叁年陸月,1275.0,5.0,1.0,4.0


In [7]:
y1_0=y1_0.sample(400, random_state=1)
y1_1=y1_1.sample(400, random_state=1)
y1_2=y1_2.sample(400, random_state=1)
y1_3=y1_3.sample(400, random_state=1)
y1_4=y1_4.sample(400, random_state=1)
df2 = pd.concat([y1_0, y1_1, y1_2, y1_3, y1_4, y1_o],axis=0, ignore_index=True)
df2

,judge_court,judge_year,judge_month,judge_index,judge_title,F_01,F_02,F_03,F_04,F_05,...,crimePredCategory,judge_crime,judge_crimeClassify,crimeCategory,Match,judge_result,judge_resultInt,judge_class,judge_class2,judge_class3
0,臺灣南投地方法院,111.0,6.0,4.0,傷害等,0.0,0.0,0.0,0.0,0.0,...,0.0,無罪,無罪,0.0,1.0,零日,0.0,0.0,0.0,0.0
1,臺灣士林地方法院,102.0,1.0,77.0,傷害等,0.0,0.0,1.0,0.0,0.0,...,0.0,無罪,無罪,0.0,1.0,零日,0.0,0.0,0.0,0.0
2,臺灣嘉義地方法院,103.0,7.0,48.0,業務過失傷害,1.0,0.0,0.0,0.0,0.0,...,0.0,無罪,無罪,0.0,1.0,零日,0.0,0.0,0.0,0.0
3,臺灣新竹地方法院,108.0,7.0,58.0,傷害,1.0,1.0,0.0,0.0,0.0,...,10.0,無罪,無罪,0.0,0.0,零日,0.0,0.0,0.0,0.0
4,臺灣臺南地方法院,111.0,8.0,48.0,家暴傷害等,1.0,0.0,0.0,0.0,0.0,...,10.0,無罪,無罪,0.0,0.0,零日,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,臺灣新北地方法院,103.0,3.0,150.0,重傷害等,1.0,0.0,0.0,0.0,0.0,...,10.0,傷害致人重傷罪,傷害致重傷,4.0,0.0,有期徒刑參年捌月,1335.0,5.0,1.0,4.0
2200,臺灣新北地方法院,103.0,5.0,187.0,家暴重傷害等,1.0,0.0,0.0,0.0,0.0,...,4.0,而致重傷,過失傷害致重傷,2.0,0.0,有期徒刑陸年捌月,2430.0,6.0,1.0,5.0
2201,臺灣新北地方法院,102.0,1.0,128.0,重傷害,0.0,0.0,0.0,1.0,0.0,...,4.0,傷害致人重傷罪,傷害致重傷,4.0,1.0,有期徒刑肆年,1460.0,5.0,1.0,4.0
2202,臺灣新北地方法院,102.0,5.0,198.0,重傷害等,1.0,0.0,1.0,0.0,0.0,...,4.0,而致重傷,過失傷害致重傷,2.0,0.0,有期徒刑叁年陸月,1275.0,5.0,1.0,4.0


In [8]:
X_list2 = ["F_01", "F_02", "F_03", "F_04", "F_06", "F_07", "F_08", "F_11", "F_13", "F_14", "crimePredCategory", "crimeCategory", "judge_class", "judge_class2", "judge_class3"]
# X_list2 = ["F_01", "F_02", "F_03", "F_04", "F_06", "F_07", "F_08", "F_11", "F_13", "F_14"]
train_data = df2.loc[:,X_list2]
train_data['label'] = train_data['crimeCategory']
train_data = ps.DataFrame(train_data)
train_data = train_data.to_spark()
train_data.columns

/opt/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/spark-3.3.1-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/opt/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


['F_01',
 'F_02',
 'F_03',
 'F_04',
 'F_06',
 'F_07',
 'F_08',
 'F_11',
 'F_13',
 'F_14',
 'crimePredCategory',
 'crimeCategory',
 'judge_class',
 'judge_class2',
 'judge_class3',
 'label']

In [9]:
feature_cols = train_data.columns[:-5]
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
train_data = assembler.transform(train_data)
train_data

DataFrame[F_01: double, F_02: double, F_03: double, F_04: double, F_06: double, F_07: double, F_08: double, F_11: double, F_13: double, F_14: double, crimePredCategory: double, crimeCategory: double, judge_class: double, judge_class2: double, judge_class3: double, label: double, features: vector]

In [10]:
train_model = train_data.select(['features', 'label'])
train_model.show(10)
train_model

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(11,[4,5,6,8],[1....|  0.0|
|(11,[2,5,6,8],[1....|  0.0|
|(11,[0,4,8],[1.0,...|  0.0|
|(11,[0,1,10],[1.0...|  0.0|
|(11,[0,4,9,10],[1...|  0.0|
|(11,[6,10],[1.0,2...|  0.0|
|(11,[5,6,7,8,10],...|  0.0|
|(11,[8,10],[2.0,2...|  0.0|
|(11,[0,1,5,8,10],...|  0.0|
|(11,[2,5,6,10],[1...|  0.0|
+--------------------+-----+
only showing top 10 rows



DataFrame[features: vector, label: double]

In [11]:
df3 = df.sample(650, random_state=100)
test_data = df3.loc[:, X_list2]
test_data['label'] = test_data['crimeCategory']
test_data = ps.DataFrame(test_data)
test_data = test_data.to_spark()
feature_cols2 = test_data.columns[:-5]
assembler = VectorAssembler(inputCols=feature_cols2, outputCol='features')
test_data = assembler.transform(test_data)
test_model = test_data.select(['features', 'label'])
test_model.show(10)

/opt/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/spark-3.3.1-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/opt/spark-3.3.1-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+--------------------+-----+
|            features|label|
+--------------------+-----+
|(11,[8,10],[2.0,9...|    1|
|(11,[1,8,10],[1.0...|    1|
|(11,[0,4,8,10],[1...|    1|
|     (11,[10],[9.0])|    3|
|(11,[0,1,4,5,8,10...|    1|
|(11,[8,10],[2.0,2...|    1|
|    (11,[10],[10.0])|    3|
|(11,[1,10],[2.0,9...|    1|
|(11,[1,8,10],[2.0...|    1|
|(11,[1,10],[1.0,9...|    1|
+--------------------+-----+
only showing top 10 rows



In [12]:
%%time
# T1=time.time()
S_RFC = RandomForestClassifier(labelCol="label", numTrees=2000, maxDepth=6, impurity='entropy')

RFC_model = S_RFC.fit(train_model)

y_pred_RFC = RFC_model.transform(test_model)
y_pred_RFC.show(10)

evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

accuracy_RFC = evaluator.evaluate(y_pred_RFC)
print(f'accuracy:{accuracy_RFC*100:.2f}%')
# T2=time.time()
# print(f"總花費時間 : {(T2 - T1):.2f} ")

23/01/06 16:22:28 WARN DAGScheduler: Broadcasting large task binary with size 1317.2 KiB
23/01/06 16:22:29 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


23/01/06 16:22:30 WARN DAGScheduler: Broadcasting large task binary with size 1340.9 KiB


23/01/06 16:22:30 WARN DAGScheduler: Broadcasting large task binary with size 7.2 MiB


23/01/06 16:22:32 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


23/01/06 16:22:32 WARN DAGScheduler: Broadcasting large task binary with size 14.9 MiB


...

23/01/06 16:22:35 WARN DAGScheduler: Broadcasting large task binary with size 28.6 MiB


23/01/06 16:22:37 WARN DAGScheduler: Broadcasting large task binary with size 7.9 MiB


23/01/06 16:22:39 WARN DAGScheduler: Broadcasting large task binary with size 27.3 MiB


+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(11,[8,10],[2.0,9...|    1|[148.097154591335...|[0.07404857729566...|       3.0|
|(11,[1,8,10],[1.0...|    1|[99.3310505035992...|[0.04966552525179...|       1.0|
|(11,[0,4,8,10],[1...|    1|[117.336409042947...|[0.05866820452147...|       1.0|
|     (11,[10],[9.0])|    3|[219.528214030254...|[0.10976410701512...|       3.0|
|(11,[0,1,4,5,8,10...|    1|[84.3103144161435...|[0.04215515720807...|       1.0|
|(11,[8,10],[2.0,2...|    1|[129.267946596699...|[0.06463397329834...|       1.0|
|    (11,[10],[10.0])|    3|[295.126977308217...|[0.14756348865410...|       3.0|
|(11,[1,10],[2.0,9...|    1|[108.888127286775...|[0.05444406364338...|       1.0|
|(11,[1,8,10],[2.0...|    1|[93.4588378662065...|[0.04672941893310...|       1.0|
|(11,[1,10],[1.0

accuracy:83.85%
CPU times: user 44.6 ms, sys: 8.9 ms, total: 53.5 ms
Wall time: 17.7 s


In [13]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [14]:
T1=time.time()

paramGrid = (ParamGridBuilder()
             .addGrid(S_RFC.maxDepth,[6,10])
             .build())

cv = CrossValidator(estimator=S_RFC, 
                    estimatorParamMaps=paramGrid, 
                    evaluator=evaluator, 
                    numFolds=5)

cv_model = cv.fit(train_model)

best_rf_model = cv_model.bestModel
model_predictions = best_rf_model.transform(test_model)
rf_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
rf_accuracy = rf_evaluator.evaluate(model_predictions)
print(rf_accuracy)
print({param.name: value for param, value in zip(best_rf_model.extractParamMap().keys(), best_rf_model.extractParamMap().values())})
T2=time.time()
print(f"總花費時間 : {(T2 - T1):.2f} ")

23/01/06 16:22:48 WARN DAGScheduler: Broadcasting large task binary with size 1327.2 KiB
23/01/06 16:22:49 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
23/01/06 16:22:49 WARN DAGScheduler: Broadcasting large task binary with size 1341.2 KiB
23/01/06 16:22:49 WARN DAGScheduler: Broadcasting large task binary with size 7.2 MiB


23/01/06 16:22:50 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


23/01/06 16:22:51 WARN DAGScheduler: Broadcasting large task binary with size 14.9 MiB


23/01/06 16:22:52 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB


23/01/06 16:22:53 WARN DAGScheduler: Broadcasting large task binary with size 28.4 MiB


...

23/01/06 16:24:33 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_0 !
23/01/06 16:24:33 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_6 !
23/01/06 16:24:33 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_2 !
23/01/06 16:24:33 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_0 !
23/01/06 16:24:33 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_4 !
23/01/06 16:24:33 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_2 !
23/01/06 16:24:33 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_4 !
23/01/06 16:24:33 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_6 !
23/01/06 16:24:33 ERROR YarnScheduler: Lost executor 3 on dtw3: Container from a bad node: container_1672985364535_0009_01_000004 on host: dtw3. Exit status: 143. Diagnostics: [2023-01-06 16:24:33.582]Container killed on request. Exit code is 143
[2023

23/01/06 16:24:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_7 !
23/01/06 16:24:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_5 !
23/01/06 16:24:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_7 !
23/01/06 16:24:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_3 !
23/01/06 16:24:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_1 !
23/01/06 16:24:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_5 !
23/01/06 16:24:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_3 !
23/01/06 16:24:39 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_1 !
23/01/06 16:24:39 WARN TransportChannelHandler: Exception in connection from /192.168.31.219:51760
java.io.IOException: Connection reset by peer
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatche

23/01/06 16:25:07 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_7 !
23/01/06 16:25:07 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_5 !
23/01/06 16:25:07 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_7 !
23/01/06 16:25:07 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_3 !
23/01/06 16:25:07 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_1 !
23/01/06 16:25:07 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_5 !
23/01/06 16:25:07 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_3 !
23/01/06 16:25:07 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_1 !
23/01/06 16:25:07 ERROR YarnScheduler: Lost executor 5 on dtw3: Container from a bad node: container_1672985364535_0009_02_000002 on host: dtw3. Exit status: 143. Diagnostics: [2023-01-06 16:25:08.057]Container killed on request. Exit code is 143
[2023

23/01/06 16:25:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_0 !
23/01/06 16:25:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_6 !
23/01/06 16:25:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_2 !
23/01/06 16:25:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_0 !
23/01/06 16:25:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_4 !
23/01/06 16:25:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_166_2 !
23/01/06 16:25:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_4 !
23/01/06 16:25:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_6 !
23/01/06 16:25:11 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 6 for reason Container from a bad node: container_1672985364535_0009_02_000003 on host: dtw2. Exit status: 143. Diagnostics: [2023-01-06 16:25:11.26

Py4JJavaError: An error occurred while calling o720.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 68.0 failed 4 times, most recent failure: Lost task 4.3 in stage 68.0 (TID 481) (dtw2 executor 6): ExecutorLostFailure (executor 6 exited caused by one of the running tasks) Reason: Container from a bad node: container_1672985364535_0009_02_000003 on host: dtw2. Exit status: 143. Diagnostics: [2023-01-06 16:25:11.265]Container killed on request. Exit code is 143
[2023-01-06 16:25:11.265]Container exited with a non-zero exit code 143. 
[2023-01-06 16:25:11.266]Killed by external signal
.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(Resizabl

In [14]:
T1=time.time()

paramGrid = (ParamGridBuilder()
             .addGrid(S_RFC.maxDepth,[6,8,10,15,20,30])
             .addGrid(S_RFC.numTrees,[100,500,1000,2000,5000,10000])
             .addGrid(S_RFC.impurity,['gini','entropy'])
             .build())

cv = CrossValidator(estimator=S_RFC, 
                    estimatorParamMaps=paramGrid, 
                    evaluator=evaluator, 
                    numFolds=5)

cv_model = cv.fit(train_model)

best_rf_model = cv_model.bestModel
model_predictions = best_rf_model.transform(test_model)
rf_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
rf_accuracy = rf_evaluator.evaluate(model_predictions)
print(rf_accuracy)
print({param.name: value for param, value in zip(best_rf_model.extractParamMap().keys(), best_rf_model.extractParamMap().values())})
T2=time.time()
print(f"總花費時間 : {(T2 - T1):.2f} ")

23/01/06 16:07:34 WARN DAGScheduler: Broadcasting large task binary with size 1468.7 KiB
23/01/06 16:07:35 WARN DAGScheduler: Broadcasting large task binary with size 1451.4 KiB
23/01/06 16:07:36 WARN DAGScheduler: Broadcasting large task binary with size 1531.3 KiB
23/01/06 16:07:36 WARN DAGScheduler: Broadcasting large task binary with size 1484.5 KiB
23/01/06 16:07:37 WARN DAGScheduler: Broadcasting large task binary with size 1893.2 KiB
23/01/06 16:07:37 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
23/01/06 16:07:38 WARN DAGScheduler: Broadcasting large task binary with size 1148.2 KiB
23/01/06 16:07:38 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
23/01/06 16:07:38 WARN DAGScheduler: Broadcasting large task binary with size 1821.0 KiB
23/01/06 16:07:39 WARN DAGScheduler: Broadcasting large task binary with size 6.6 MiB
23/01/06 16:07:40 WARN DAGScheduler: Broadcasting large task binary with size 1900.1 KiB
23/01/06 16:07:40 WARN DAGSche

23/01/06 16:07:50 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB


23/01/06 16:07:50 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
23/01/06 16:07:52 WARN DAGScheduler: Broadcasting large task binary with size 1327.1 KiB
23/01/06 16:07:52 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
23/01/06 16:07:53 WARN DAGScheduler: Broadcasting large task binary with size 1336.5 KiB
23/01/06 16:07:53 WARN DAGScheduler: Broadcasting large task binary with size 7.2 MiB


23/01/06 16:07:54 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


23/01/06 16:07:54 WARN DAGScheduler: Broadcasting large task binary with size 14.7 MiB


...

23/01/06 16:10:56 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_769_5 !
23/01/06 16:10:56 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_7 !
23/01/06 16:10:56 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_769_7 !
23/01/06 16:10:56 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_769_3 !
23/01/06 16:10:56 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_3 !
23/01/06 16:10:56 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_769_1 !
23/01/06 16:10:56 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_1 !
23/01/06 16:10:56 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_5 !
23/01/06 16:10:56 ERROR YarnScheduler: Lost executor 7 on dtw4: Container from a bad node: container_1672985364535_0008_02_000004 on host: dtw4. Exit status: 143. Diagnostics: [2023-01-06 16:10:56.624]Container killed on request. Exit code is 143
[2023

Py4JJavaError: An error occurred while calling o3360.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 249.0 failed 4 times, most recent failure: Lost task 3.3 in stage 249.0 (TID 1734) (dtw4 executor 7): ExecutorLostFailure (executor 7 exited caused by one of the running tasks) Reason: Container from a bad node: container_1672985364535_0008_02_000004 on host: dtw4. Exit status: 143. Diagnostics: [2023-01-06 16:10:56.624]Container killed on request. Exit code is 143
[2023-01-06 16:10:56.624]Container exited with a non-zero exit code 143. 
[2023-01-06 16:10:56.624]Killed by external signal
.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(Resi

23/01/06 16:10:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_769_4 !
23/01/06 16:10:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_769_0 !
23/01/06 16:10:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_2 !
23/01/06 16:10:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_0 !
23/01/06 16:10:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_769_6 !
23/01/06 16:10:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_769_2 !
23/01/06 16:10:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_4 !
23/01/06 16:10:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_65_6 !


23/01/06 16:10:58 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.31.51:52068 is closed
23/01/06 16:10:58 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to get executor loss reason for executor id 8 at RPC address 192.168.31.226:45134, but got no response. Marking as agent lost.
java.io.IOException: Connection from /192.168.31.51:52068 closed
	at org.apache.spark.network.client.TransportResponseHandler.channelInactive(TransportResponseHandler.java:147)
	at org.apache.spark.network.server.TransportChannelHandler.channelInactive(TransportChannelHandler.java:117)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:262)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelInactive(AbstractChannelHandlerContext.java:248)
	at io.netty.channel.AbstractChannelHandlerContext.fireChannelInactive(AbstractChannelHandlerContext.java:241)
	at io.netty.channel.ChannelInboundH

23/01/06 16:10:58 ERROR YarnClientSchedulerBackend: YARN application has exited unexpectedly with state FAILED! Check the YARN application logs for more details.
23/01/06 16:10:58 ERROR YarnClientSchedulerBackend: Diagnostics message: Max number of executor failures (3) reached
23/01/06 16:10:58 ERROR TransportClient: Failed to send RPC RPC 7463134832591099929 to /192.168.31.51:52068: io.netty.channel.StacklessClosedChannelException
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source)
23/01/06 16:10:58 ERROR YarnSchedulerBackend$YarnSchedulerEndpoint: Sending RequestExecutors(Map(),Map(),Map(),Set()) to AM was unsuccessful
java.io.IOException: Failed to send RPC RPC 7463134832591099929 to /192.168.31.51:52068: io.netty.channel.StacklessClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.java:392)
	at org.apache.spark.netwo